In [84]:
from IPython.display import Image
import ee
import shapefile
import numpy as np
ee.Initialize()

In [85]:
x1=88.00
y1=21.50
x2=90.00
y2=22.70
polygon1 = [[x1,y1],[x2,y1],[x2,y2],[x1,y2],[x1,y1]]
#poly1=ee.Geometry.Polygon(polygon1)
#feat1=ee.FeatureCollection(poly1)
sf = shapefile.Reader("BGD_adm/BGD_adm3.shp")
shape_ex=sf.shapes()[277]
lon_lat = []
n=len(shape_ex.points)
print(len(sf.shapes()),n)
i=0
for ip in range(n):
    #i=i+1
    #if i==100:
        i=0
        lon_lat.append([shape_ex.points[ip][0],shape_ex.points[ip][1]])
#lon_lat[-1]=lon_lat[0]
#print(len(lon_lat))
polygon=polygon1
#f=open('data.csv','w')
#for l in lon_lat:
    #f.write(str(l[0])+','+str(l[1]))
    #f.write('\n')
#f.close()
poly=ee.Geometry.Polygon(polygon)
feat=ee.FeatureCollection(poly)
#print(lon_lat)

463 2780


In [86]:
import pandas as pd
# Input data
#area_of_interest = ee.FeatureCollection('ft:1ihcmnTQF2dUYTKXOIYYwYlJzFLSpO7zsIxg0Yqd5')
area_of_interest = ee.FeatureCollection(poly)
#mexico_landcover_2010_landsat = ee.Image("LANDSAT/LE07/C01/T1").clip(area_of_interest)
landsat8_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')#('LANDSAT/LC8_L1T_TOA')
landsat8_collection_dt=landsat8_collection.filterDate('2013-01-01', '2018-10-31')
landsat8_collection_pl=landsat8_collection_dt.filterBounds(poly)
final_coll=landsat8_collection_pl.filter(ee.Filter.lt('CLOUD_COVER', 5))
#final_coll=final_coll_1.filterMetadata('IMAGE_QUALITY', 'equals', 2);
#landsat8_collection_2 = landsat8_collection_pl.min().slice(0,9)
point = {'type':'Point', 'coordinates':[92, 23]};
info=final_coll.getInfo()

lst_img=final_coll.toList(final_coll.size())
images=[]

n=(final_coll.size().getInfo())
names=[]
for i in range(n):
        images.append(ee.Image(lst_img.get(i)))
        inf=images[i].getInfo()
        id_img=inf['id']
        name=id_img.split('/')[-1]
        names.append(name)
n=len(images)
print(n)
print(names[:])

196
['LC08_137044_20131224', 'LC08_137044_20140125', 'LC08_137044_20140210', 'LC08_137044_20140226', 'LC08_137044_20140330', 'LC08_137044_20141008', 'LC08_137044_20141024', 'LC08_137044_20141125', 'LC08_137044_20150128', 'LC08_137044_20150213', 'LC08_137044_20150317', 'LC08_137044_20151027', 'LC08_137044_20151112', 'LC08_137044_20160115', 'LC08_137044_20160216', 'LC08_137044_20161114', 'LC08_137044_20161130', 'LC08_137044_20161216', 'LC08_137044_20170117', 'LC08_137044_20170202', 'LC08_137044_20170218', 'LC08_137044_20170322', 'LC08_137044_20171203', 'LC08_137044_20180120', 'LC08_137044_20180221', 'LC08_137044_20180512', 'LC08_137044_20181003', 'LC08_137045_20131224', 'LC08_137045_20140109', 'LC08_137045_20140125', 'LC08_137045_20140226', 'LC08_137045_20140330', 'LC08_137045_20140415', 'LC08_137045_20141008', 'LC08_137045_20141024', 'LC08_137045_20141125', 'LC08_137045_20150112', 'LC08_137045_20150128', 'LC08_137045_20150317', 'LC08_137045_20151027', 'LC08_137045_20151112', 'LC08_13704

In [87]:
root='sb_2013_18/'

In [88]:
f=open(root+'names.csv','w')
for name in names:   
    f.write(name)
    f.write('\n')
f.close()
print(name)

LC08_139045_20181001


In [89]:
#date=name.split('_')[-1]
#year=date[:4]
#month=date[4:6]
#day=date[6:]
#print(name,year,month,day)
#print(inf.keys())
#print(inf['version'])-

In [90]:
# Functions to derive vegetation indices and other raster operations
def NDVI(image):
    return image.normalizedDifference(['B5', 'B4'])

def SAM(image):
    band1 = image.select("B1")
    bandn = image.select("B2","B3","B4","B5","B6","B7","B8","B9");
    maxObjSize = 256;
    b = band1.divide(bandn);
    spectralAngleMap = b.atan();
    spectralAngleMap_sin = spectralAngleMap.sin();
    spectralAngleMap_cos = spectralAngleMap.cos();
    sum_cos = spectralAngleMap_cos.reduce(ee.call("Reducer.sum"));
    sum_sin = spectralAngleMap_sin.reduce(ee.call("Reducer.sum"));
    return ee.Image.cat(sum_sin, sum_cos, spectralAngleMap_sin, spectralAngleMap_cos);

#Enhanced Vegetation Index
def EVI(image):
    # L(Canopy background)
    # C1,C2(Coefficients of aerosol resistance term)
    # GainFactor(Gain or scaling factor)
    gain_factor = ee.Image(2.5);
    coefficient_1 = ee.Image(6);
    coefficient_2 = ee.Image(7.5);
    l = ee.Image(1);
    nir = image.select("B5");
    red = image.select("B4");
    blue = image.select("B2");
    evi = image.expression(
        "Gain_Factor*((NIR-RED)/(NIR+C1*RED-C2*BLUE+L))",
        {
            "Gain_Factor":gain_factor,
            "NIR":nir,
            "RED":red,
            "C1":coefficient_1,
            "C2":coefficient_2,
            "BLUE":blue,
            "L":l
        }
    )
    return evi

#Atmospherically Resistant Vegetation Index
def ARVI(image):
    red = image.select("B4")
    blue = image.select("B2")
    nir = image.select("B5")
    red_square = red.multiply(red)
    arvi = image.expression(
        "NIR - (REDsq - BLUE)/(NIR+(REDsq-BLUE))",{
            "NIR": nir,
            "REDsq": red_square,
            "BLUE": blue
        }
    )
    return arvi

#Leaf Area Index
def LAI(image):
    nir = image.select("B5")
    red = image.select("B4")
    coeff1 = ee.Image(0.0305);
    coeff2 = ee.Image(1.2640);
    lai = image.expression(
        "(((NIR/RED)*COEFF1)+COEFF2)",
        {
            "NIR":nir,
            "RED":red,
            "COEFF1":coeff1,
            "COEFF2":coeff2
        }
    )
    return lai

def tasseled_cap_transformation(image):
    #Tasseled Cap Transformation for Landsat 8 based on the 
    #scientfic work "Derivation of a tasselled cap transformation based on Landsat 8 at-satellite reflectance" 
    #by M.Baigab, L.Zhang, T.Shuai & Q.Tong (2014). The bands of the output image are the brightness index, 
    #greenness index and wetness index.
    b = image.select("B2", "B3", "B4", "B5", "B6", "B7");
    #Coefficients are only for Landsat 8 TOA
    brightness_coefficents= ee.Image([0.3029, 0.2786, 0.4733, 0.5599, 0.508, 0.1872])
    greenness_coefficents= ee.Image([-0.2941, -0.243, -0.5424, 0.7276, 0.0713, -0.1608]);
    wetness_coefficents= ee.Image([0.1511, 0.1973, 0.3283, 0.3407, -0.7117, -0.4559]);
    fourth_coefficents= ee.Image([-0.8239, 0.0849, 0.4396, -0.058, 0.2013, -0.2773]);
    fifth_coefficents= ee.Image([-0.3294, 0.0557, 0.1056, 0.1855, -0.4349, 0.8085]);
    sixth_coefficents= ee.Image([0.1079, -0.9023, 0.4119, 0.0575, -0.0259, 0.0252]);
    
    #Calculate tasseled cap transformation
    brightness = image.expression(
        '(B * BRIGHTNESS)',
        {
            'B':b,
            'BRIGHTNESS': brightness_coefficents
        })
    greenness = image.expression(
        '(B * GREENNESS)',
        {
            'B':b,
            'GREENNESS': greenness_coefficents
        })
    wetness = image.expression(
        '(B * WETNESS)',
        {
            'B':b,
            'WETNESS': wetness_coefficents
        })
    fourth = image.expression(
        '(B * FOURTH)',
        {
            'B':b,
            'FOURTH': fourth_coefficents
        })
    fifth = image.expression(
        '(B * FIFTH)',
        {
            'B':b,
            'FIFTH': fifth_coefficents
        })
    sixth = image.expression(
        '(B * SIXTH)',
        {
            'B':b,
            'SIXTH': sixth_coefficents
        })
    bright = brightness.reduce(ee.call("Reducer.sum"));
    green = greenness.reduce(ee.call("Reducer.sum"));
    wet = wetness.reduce(ee.call("Reducer.sum"));
    four = fourth.reduce(ee.call("Reducer.sum"));
    five = fifth.reduce(ee.call("Reducer.sum"));
    six = sixth.reduce(ee.call("Reducer.sum"));
    tasseled_cap = ee.Image(bright).addBands(green).addBands(wet).addBands(four).addBands(five).addBands(six)
    return tasseled_cap.rename('brightness','greenness','wetness','fourth','fifth','sixth')

In [91]:
import requests,urllib
for i in range(n):
    image=images[i]
    #tct = tasseled_cap_transformation(images[i])
    #tct_g=tct.select('greenness')
    #image = image_a.clip(feat)
    path = image.getDownloadUrl({
        'scale': 500,
        'crs': 'EPSG:4326'#WGS84
        #'region':polygon
        })
    print(path)
    res=urllib.request.urlopen(path)
    f=open(root+str(i)+'.zip','wb')
    f.write(res.read())
    f.close()
#request = urllib2.Request(self.url)
#pic = urllib2.urlopen(request)



https://earthengine.googleapis.com/api/download?docid=f1b50dd1eea2b2b183f4b9b9c687199a&token=9320e801d5cb3a9f0e8127e92321528c
https://earthengine.googleapis.com/api/download?docid=76575d2dc55737fcc50e551f1b1b67ae&token=c36b4af3aa06d88ecd2e3bcda8e950f6
https://earthengine.googleapis.com/api/download?docid=4e3ce70615358bbdf2fb328bc129d657&token=e73d9d835f299c89a73d4ce5a76c857d
https://earthengine.googleapis.com/api/download?docid=8f2720e7db694612657d5a9a1837dee9&token=98bb9efe63ab2dc606f6f8d1ebe37967
https://earthengine.googleapis.com/api/download?docid=8767212d20599b1ee2cc5d443c125fc0&token=bca6a1cac8aefe76a073c90ab3972f15
https://earthengine.googleapis.com/api/download?docid=dbdf590c7f7a13c24cfb17e077e5d444&token=0df04e96ca08a0084fbac1347f5e525f
https://earthengine.googleapis.com/api/download?docid=86fd74534b9338845b7997ded2419407&token=96e0aa85dedf1f26ef3d3d438b03ff8c
https://earthengine.googleapis.com/api/download?docid=31626883139eb671d3b0bf091f074903&token=5e430d6a915cb6f0e888956e6

https://earthengine.googleapis.com/api/download?docid=70e5cfc2ee76811bd85a719afb2db1df&token=70e767d827ed45f874f7b5819d7d6fd1
https://earthengine.googleapis.com/api/download?docid=8c1d1aabbd2d9f9832b1df1a0d8e6f69&token=0c47d58308c002183d43a168461c89db
https://earthengine.googleapis.com/api/download?docid=b88ac2296f878172c0887b1ed19ef8a2&token=d8bc88702a660f1eac45b12efb8cb18b
https://earthengine.googleapis.com/api/download?docid=44f5ea7ede98c9db4dbc0217dcd76f9a&token=856c8346492481bff20fa0faf11ee6fb
https://earthengine.googleapis.com/api/download?docid=98957f622a9ba99859da8d00b68e16b9&token=e9ac48f5a86ba7b6d4c108856829b399
https://earthengine.googleapis.com/api/download?docid=479abf4ca2be3c3bdc56fbab970c010e&token=8d98b9d3f11a02b11e205e7563fe6577
https://earthengine.googleapis.com/api/download?docid=3ce7fcf53bc9e6b9a6488f50b2b88fdd&token=4625dd18ec8fdc9ff8f8a3f829a5d27f
https://earthengine.googleapis.com/api/download?docid=e0331f8201a2397e0b6a3d6317bc9d53&token=9aeadfb2f1aa0eb486c497cc5

https://earthengine.googleapis.com/api/download?docid=35f7e2ed413d05ffae61af1f28500613&token=c92cf9fcf4042ced4204367c310f78cd
https://earthengine.googleapis.com/api/download?docid=053e0498004183578223a86e062cac9d&token=c97afccadbaa388faee5280439eda6a6
https://earthengine.googleapis.com/api/download?docid=db7b027c2c4ce3115c402a14e8bd60cd&token=eb77cec2b88a263a9ca1367a9bfcd992
https://earthengine.googleapis.com/api/download?docid=8b8f15ae969d6270223b7d2d44541c4f&token=34672ac9de2ba821cd5a4f220d65cdc9
https://earthengine.googleapis.com/api/download?docid=ac3148654aec858997827c547cc3c8bb&token=db893f6589a153a542f988d9397ea4a4
https://earthengine.googleapis.com/api/download?docid=59d12a1549895f99bdf149af653939eb&token=83abb678446ac32125b7229bdb80a59d
https://earthengine.googleapis.com/api/download?docid=cc222d5526f8b974b281ca271326c526&token=7fa7b60e8218dc8a66769d5a47d4b2b7
https://earthengine.googleapis.com/api/download?docid=4a983dac3613c97dfbb214072a97cbb5&token=de0ce6ea5a62908e7c40af6ca

In [92]:
import zipfile
for i in range(n):
    print(i)
    with zipfile.ZipFile(root+str(i)+'.zip', 'r') as zip_ref:
        zip_ref.extractall(root)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195


In [93]:
import csv
f=open(root+'names.csv','r')
lines =f.read()
names=lines.split('\n')
f.close()
print(names[0])

LC08_137044_20131224


In [94]:
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np


In [95]:
for name in names:
    blue=Image.open(root+name+'.B2.tif')
    blue=np.array(blue,dtype=np.float64)
    green=Image.open(root+name+'.B3.tif')
    green=np.array(green,dtype=np.float)
    red=Image.open(root+name+'.B4.tif')
    red=np.array(red,dtype=np.float)
    sp=blue.shape
    img=np.zeros((sp[0],sp[1],3),dtype=np.float64)
    img[:,:,0]=blue
    img[:,:,1]=green
    img[:,:,2]=red
    img[img<-500]=0
    img=np.uint8(img*1000)
    #plt.imshow(img)
    #plt.show()
    cv2.imwrite(root+'img/'+name+'.jpg',img)
    print(name)
print(img.max())

LC08_137044_20131224
LC08_137044_20140125
LC08_137044_20140210
LC08_137044_20140226
LC08_137044_20140330
LC08_137044_20141008
LC08_137044_20141024
LC08_137044_20141125
LC08_137044_20150128
LC08_137044_20150213
LC08_137044_20150317
LC08_137044_20151027
LC08_137044_20151112
LC08_137044_20160115
LC08_137044_20160216
LC08_137044_20161114
LC08_137044_20161130
LC08_137044_20161216
LC08_137044_20170117
LC08_137044_20170202
LC08_137044_20170218
LC08_137044_20170322
LC08_137044_20171203
LC08_137044_20180120
LC08_137044_20180221
LC08_137044_20180512
LC08_137044_20181003
LC08_137045_20131224
LC08_137045_20140109
LC08_137045_20140125
LC08_137045_20140226
LC08_137045_20140330
LC08_137045_20140415
LC08_137045_20141008
LC08_137045_20141024
LC08_137045_20141125
LC08_137045_20150112
LC08_137045_20150128
LC08_137045_20150317
LC08_137045_20151027
LC08_137045_20151112
LC08_137045_20151128
LC08_137045_20160115
LC08_137045_20160131
LC08_137045_20160216
LC08_137045_20161114
LC08_137045_20161130
LC08_137045_2

FileNotFoundError: [Errno 2] No such file or directory: 'sb_2013_18/.B2.tif'

In [7]:
#Display thumbnail of classification output
#image=landsat8_collection_21

#theGeom = area_of_interest.geometry().getInfo()['coordinates']
for i in range(2):
    image=images[i]
    #image=image.select('B2','B3','B4')
   # image = image_a.clip(feat)
    thumbnail = image.getThumbUrl({'min':-1,'max':1,'size':'800','region':polygon})
    fname='ls_bd/'+'toa_l8'+str(i)+'.jpg'
    print(thumbnail)
    res=urllib.request.urlretrieve(thumbnail,fname)
    print(res)
    Image(url=thumbnail)

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
import requests,urllib
#Display thumbnail of classification output
#image=landsat8_collection_2
#image = landsat8_collection.clip(area_of_interest)
#theGeom = area_of_interest.geometry().getInfo()['coordinates']
for i in range(n):
    image=images[i]
    tct = tasseled_cap_transformation(image)
    tct_green=tct.select('wetness')
    thumbnail = tct_green.getThumbUrl({'min':-1,'max':1,'size':'800','region':polygon})
    #Image(url=thumbnail)
    fname='ls_bd/'+'toa_w'+str(i)+'.jpg'
    res=urllib.request.urlretrieve(thumbnail,fname)
    print(res)


In [ ]:
tct = tasseled_cap_transformation(images[10])
tct.bandNames().getInfo()

In [ ]:

#Display thumbnail of classification output
#image=landsat8_collection_2
#image = landsat8_collection.clip(area_of_interest)
#theGeom = area_of_interest.geometry().getInfo()['coordinates']
image=tct
thumbnail = image.getThumbUrl({'min':-1,'max':1,'size':'800','region':polygon})
Image(url=thumbnail)

In [ ]:
#Derive indices, spectral angles and build an image stack
ndvi = NDVI(landsat8_collection)
sam = SAM(landsat8_collection)
evi = EVI(landsat8_collection)
arvi = ARVI(landsat8_collection)
lai = LAI(landsat8_collection)
spectral_indices_stack = ee.Image(ndvi).addBands(lai).addBands(sam).addBands(arvi).addBands(evi)

In [ ]:
#Define classification function
def classification(raster_input, training_dataset,number_of_training_points, region, classification_algorithm):
    bands = raster_input.bandNames()
    points = ee.FeatureCollection.randomPoints(region, number_of_training_points, number_of_training_points, 1)
    training = training_dataset.addBands(raster_input).reduceToVectors(
        reducer="mean",
        geometry=points,
        geometryType="centroid",
        scale=30, 
        crs="EPSG:4326"
    )

    classifier = training.trainClassifier(
        property_list=bands,
        class_property="label",
        classifier_name=classification_algorithm
    )
    # Apply the classifier to the original composite.
    out = raster_input.classify(classifier)
    return out

In [ ]:
#Execute classification function
output = classification(spectral_indices_stack, mexico_landcover_2010_landsat, 1000, area_of_interest, 'Cart')

In [ ]:
#Display thumbnail of classification output
image = output.clip(area_of_interest)
theGeom = area_of_interest.geometry().getInfo()['coordinates']
thumbnail = image.getThumbUrl({'min':0,'max':211,'size':'800','region':theGeom})
Image(url=thumbnail)